# KELOMPOK E
Anggota:
1. Nurkhalisha Humaira (2106703802)
2. Rima Fitrianti Azahra (2106701974)
3. Farah Ramadhani Putri (2106654832)
4. Daniella Putri Shalomita (2106631072)
5. Musarofah Kurnia (2106652543)

# PENDAHULUAN
## 1. Deskripsi Data
Data ini diambil dari site ASHRAE (American Society of Heating and Air-Conditioning Engineers). Data ini berisi tentang penggunaan 4 energi yang berbeda pada gedung-gedung di beberapa wilayah dalam satu tahun. Variabel yang ada dari data ini adalah: <br>

**train.csv:** 
  1. building_id - ID Gedung
  2. meter - Meter id code. Dengan Keterangan {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}.
  3. timestamp - Waktu Pengukuran
  4. meter_reading - Variabel target yaitu energi yang digunakan dalam kWh. 
  5. site_id - Foreign key untuk data cuaca.
  6. building_id - Foreign key untuk data train
  7. primary_use - Penggunaan gedung sehari-hari
  8. square_feet - luas lantai gedung
  9. year_built - Tahun gedung dibuka
  10. floor_count - jumlah lantai


**weather_[train/test].csv**: Data Cuaca dari stasiun metereologi terdekat site gedung

  1. site_id - ID site gedung
  2. air_temperature - Suhu Udara (Derajat Celsius)
  3. cloud_coverage - Jumlah awan yang menutupi langit (oktas)
  4. dew_temperature - Suhu Embun (Derajat Celsius)
  5. precip_depth_1_hr - Presipitasi Air Hujan (Millimeter)
  6. sea_level_pressure - Millibar/hectopascals
  7. wind_direction - Arah Angin berdasarkan Compass direction (0-360)
  8. wind_speed - Kecepatan angin (m/s)

**test.csv**: 

  1. row_id - Row id file submission
  2. building_id - Building id code
  3. meter - meter id code
  4. timestamp - Jangka waktu data test
  
## 2. Permasalahan
Permasalahan yang akan dianalisa adalah bagaimana melihat penggunaan 4 energi tersebut, yaitu:<br>
- Energi Listrik/Electricity (0)
- Air Dingin/Chilled Water (1)
- Uap/Steam (2)
- Air Panas/Hot Water (3)

dengan informasi gedung, cuaca, dan meter reading. Nantinya informasi ini akan digunakan untuk menentukan efisiensi energi.


# LOADING DATA
## Inisasi Data

In [ ]:
#importing packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Data Gedung
gd = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')
#Data Meter Reading
en_test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
en_train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
#Data Cuaca
cu_test = pd.read_csv('../input/ashrae-energy-prediction/weather_test.csv')
cu_train = pd.read_csv('../input/ashrae-energy-prediction/weather_train.csv')

## Menampilkan Data

In [ ]:
gd

In [ ]:
en_test

In [ ]:
en_train

In [ ]:
cu_test

In [ ]:
cu_train

In [ ]:
#melihat info masing-masing data untuk data understanding
gd.info()

en_test.info()
en_train.info()

cu_test.info()
cu_train.info()

# Reducing Memory
Karena data yang digunakan cukup besar, sering terjadi *crash* saat run program, maka akan direduce data untuk mengurangi besar memori data

In [ ]:
#fungsi rekursif untuk reducing memory
def reduce_mem_usage(df, df_name):

    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:

        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2

    print('Memory usage of {} is reduced by {:.2f} %. Usage dropped from {:.2f} MB to {:.2f} MB.'.format(df_name, (100 * (start_mem - end_mem) / start_mem), start_mem, end_mem))
    
    return df

In [ ]:
#reducing memory dataframe yang telah diinisiasi
gd = reduce_mem_usage(gd, 'Data Gedung')

en_train = reduce_mem_usage(en_train, 'Data Train')
en_test = reduce_mem_usage(en_test, 'Data Test')

cu_test = reduce_mem_usage(cu_test, "Data Cuaca Test")
cu_train = reduce_mem_usage(cu_train, 'Data Cuaca Train')

# MERGING DATA
Data akan dikelompokkan menjadi 2 Dataframe, namun EDA akan difokuskan terhadap dataframe train.

In [ ]:
train = en_train.merge(gd, on='building_id', how='left')
train = train.merge(cu_train, on=['site_id', 'timestamp'], how='left')

test = en_test.merge(gd, on='building_id', how='left')
test = test.merge(cu_test, on=['site_id', 'timestamp'], how='left')

display(train.head(), test.head())

In [ ]:
train.info()

# Melihat Statistika Deskriptif Dari Dataframe

In [ ]:
train.describe()

In [ ]:
test.describe()

# PREPROCESSING

## Mengubah dtype dan adding feature baru
Variabel timestamps masih terbaca sebagai objek, maka akan diubah menjadi datetime dan di breakdown menjadi "day", "hour", "weekday", "month", dan "year" untuk kemudahan visualisasi nantinya.

In [ ]:
import datetime as dt
train["timestamp"] = pd.to_datetime(train["timestamp"])
train["day"]       = train["timestamp"].dt.day
train["hour"]      = train["timestamp"].dt.hour
train["week"]      = train["timestamp"].dt.weekday
train["month"]     = train["timestamp"].dt.month
train["year"]      = train["timestamp"].dt.year
train              = train.drop("timestamp", axis = 1)

In [ ]:
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["day"]       = test["timestamp"].dt.day
test["hour"]      = test["timestamp"].dt.hour
test["week"]      = test["timestamp"].dt.weekday
test["month"]     = test["timestamp"].dt.month
test["year"]      = test["timestamp"].dt.year
test              = test.drop("timestamp", axis = 1)

## Mencari missing value
Akan dicari persentase missing value dari tiap-tiap variabel

### Data Train

In [ ]:
train_missing_values = pd.DataFrame(train.isnull().sum() * 100 / len(train))
train_missing_values.columns = ["Missing Values"]

train_missing_values = train_missing_values[train_missing_values["Missing Values"] != 0]
train_missing_values.sort_values(by = "Missing Values", axis = 0, ascending = False, inplace = True)
train_missing_values

In [ ]:
#Visualisasi
plt.figure(figsize=(8,8))
sns.barplot(x = train_missing_values['Missing Values'], y = train_missing_values.index).set_title("Persentase Missing Value Variabel Pada Data Train", fontsize=16)
plt.xlabel('Persentase Missing', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.show()

### Data Test

In [ ]:
test_missing_values = pd.DataFrame(test.isnull().sum() * 100 / len(test))
test_missing_values.columns = ["Missing Values"]

test_missing_values = test_missing_values[test_missing_values["Missing Values"] != 0]
test_missing_values.sort_values(by = "Missing Values", axis = 0, ascending = False, inplace = True)
test_missing_values

In [ ]:
#Visualisasi
plt.figure(figsize=(8,8))
sns.barplot(x = test_missing_values['Missing Values'], y = test_missing_values.index).set_title("Persentase Missing Value Pada Variabel Data Test", fontsize=16)
plt.xlabel('Persentase Missing', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.show()

## Melihat Distribusi
Akan dilihat distribusi dari masing-masing variabel untuk filling missing value yang ada

In [ ]:
#fungsi rekursif untuk melihat distribusi data 
sns.set_theme()

def draw_distribution_graph(dataframe, column_name):
    
    fig, ax = plt.subplots(figsize=(8, 8))
    sns.distplot(dataframe[column_name], color='purple', ax = ax).set_title("Distribution Graph for {}".format(column_name), fontsize=16)
    plt.xlabel(column_name, fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.show()

### Distribusi Dari Variabel yang memiliki Missing Value >10% Pada Data Train

In [ ]:
draw_distribution_graph(train, 'precip_depth_1_hr')

**Observasi**:<br>
Terlihat distribusinya berada di sekitar 0, sedangkan pada statistika deskriptifnya terdapat nilai 343, sehingga nilai tersebut dapat diasumsikan sebagai outlier. Maka, untuk mengisi missing value pada variabel ini akan digunakan mediannya.

### Distribusi Dari Variabel yang memiliki Missing Value >10% Pada Data Test

In [ ]:
draw_distribution_graph(test, 'precip_depth_1_hr')

**Observasi**:<br> Terlihat distribusinya berada di sekitar 0, sedangkan pada statistika deskriptifnya terdapat nilai maksimumnya adalah 597, sehingga nilai tersebut dapat diasumsikan sebagai outlier. Maka, untuk mengisi missing value pada variabel ini akan digunakan mediannya.

### Melihat letak missing value pada variabel yang missing valuesnya <5% pada data train

In [ ]:
cek_na = train[train.air_temperature.isnull()]
display(cek_na.head(), cek_na.tail(), cek_na.shape)

In [ ]:
cek_na1 = train[train.wind_speed.isnull()]
display(cek_na1.head(), cek_na1.tail(), cek_na1.shape)

In [ ]:
cek_na2 = train[train.dew_temperature.isnull()]
display(cek_na2.head(), cek_na2.tail(), cek_na2.shape)

### Melihat letak missing value pada variabel yang missing valuesnya < 5% pada data test

In [ ]:
cek_na = test[test.air_temperature.isnull()]
display(cek_na.head(), cek_na.tail(), cek_na.shape)

In [ ]:
cek_na1 = test[test.wind_speed.isnull()]
display(cek_na1.head(), cek_na1.tail(), cek_na1.shape)

In [ ]:
cek_na2 = test[test.dew_temperature.isnull()]
display(cek_na2.head(), cek_na2.tail(), cek_na2.shape)

### Mengatasi missing value pada data train
karena variabel "floor_count" dan "year_built" mengandung sekitar 80% dan 60% missing value, maka kedua kolom ini akan di drop, karena jika diisi akan menyebabkan data tidak akurat karena besarnya >50%, selain itu, kedua variabel tersebut juga tidak berpengaruh terhadap variabel target meter_reading.

In [ ]:
threshold = len(train) * 0.5
train.dropna(axis=1, thresh = threshold, inplace = True)

Untuk variabel lainnya, missing value akan diimputasi dengan nilai sesuai dengan pengamatan distribusi yang telah dilakukan sebelumnya.
Untuk cloud_coverage, sea_level_pressure, dan wind_direction telah terlihat distribusinya pada statistika deskriptif, dan distribusinya tidak terlalu jauh antara nilai maksimum dan minimumnya, maka missing valuenya akan diisi dengan mediannya.
Untuk precip_depth_1_hr juga akan diisi dengan mediannya sesuai dengan observasi yang telah dilakukan di atas.

In [ ]:
train['cloud_coverage'].fillna(train['cloud_coverage'].median(), inplace=True)
train['sea_level_pressure'].fillna(train['sea_level_pressure'].median(), inplace=True)
train['precip_depth_1_hr'].fillna(train['precip_depth_1_hr'].median(), inplace=True)
train['wind_direction'].fillna(train['wind_direction'].median(), inplace=True)

Untuk variabel yang missing valuenya < 5%, berdasarkan observasi di atas, imputasi dapat dilakukan dengan dropna karena missing value terjadi karena ketiadaan data di waktu itu (terlihat dari banyaknya kolom yang juga kosong)

In [ ]:
train = train.dropna(subset=['air_temperature'])
train = train.dropna(subset=['wind_speed'])
train = train.dropna(subset=['dew_temperature'])

In [ ]:
#akan dilihat missing value yang tersisa
train.isnull().sum()

### Mengatasi missing value pada data test
karena variabel "floor_count" dan "year_built" mengandung sekitar 80% dan 60% missing value, maka kedua kolom ini akan di drop, karena jika diisi akan menyebabkan data tidak akurat karena besarnya >50%

In [ ]:
threshold = len(test) * 0.5
test.dropna(axis=1, thresh = threshold, inplace = True)

Untuk variabel lainnya, missing value akan diimputasi dengan nilai sesuai dengan pengamatan distribusi yang telah dilakukan sebelumnya.
Untuk cloud_coverage, sea_level_pressure, dan wind_direction telah terlihat distribusinya pada statistika deskriptif, dan distribusinya tidak terlalu jauh antara nilai maksimum dan minimumnya, maka missing valuenya akan diisi dengan mediannya.
Untuk precip_depth_1_hr juga akan diisi dengan mediannya sesuai dengan observasi yang telah dilakukan di atas.

In [ ]:
test['cloud_coverage'].fillna(test['cloud_coverage'].median(), inplace=True)
test['precip_depth_1_hr'].fillna(test['precip_depth_1_hr'].median(), inplace=True)
test['sea_level_pressure'].fillna(test['sea_level_pressure'].median(), inplace=True)
test['wind_direction'].fillna(test['wind_direction'].median(), inplace=True)

Untuk variabel yang missing valuenya < 5%, berdasarkan observasi di atas, imputasi dapat dilakukan dengan dropna karena missing value terjadi karena ketiadaan data di waktu itu (terlihat dari banyaknya kolom yang juga kosong)

In [ ]:
test = test.dropna(subset=['air_temperature'])
test = test.dropna(subset=['wind_speed'])
test = test.dropna(subset=['dew_temperature'])

In [ ]:
#akan dilihat missing value yangtersisa
test.isnull().sum()

# OUTLIER
Karena dataframe yang akan dipakai untuk analisis adalah dataframe train, outlier pada dataframe test dapat diabaikan.

Pertama-tama akan dilihat boxplot dari masing-masing variabel untuk dilihat persebarannya dan mendeteksi outliernya.

In [ ]:
plt.figure(figsize=(12,8))

for i, col in enumerate(train.describe().columns):
    plt.subplot(4,5, i+1)
    sns.boxplot(x=train[col])
    plt.tight_layout()
    
plt.show()

Dari boxplot, terlihat bahwa masih terdapat banyak outlier. Yang akan diatasi adalah outlier pada meter_reading karena variabel tersebut menjadi fokus (target value) pada analisa ini. Sebelum itu akan dilihat distribusi dari variabel meter_reading.

In [ ]:
draw_distribution_graph(train, 'meter_reading')

Terlihat bahwa distribusinya terlalu condong ke kiri (tidak normal). Hal ini disebabkan karena outlier tersebut, maka dari itu untuk menormalkan distribusinya, akan digunakan metode log transformation. Hal ini dilakukan untuk mengurangi "influence" outlier tersebut terhadap analisa data nantinya.

In [ ]:
train['meter_reading']=np.log1p(train['meter_reading'])
draw_distribution_graph(train, 'meter_reading')

terlihat bahwa meter_reading sudah lebih "normal" dan efek dari outlier tersebut telah tereduksi. Untuk lebih meyakinkan, akan dicek visualisasi boxplotnya.

In [ ]:
sns.boxplot(x=train['meter_reading'])

# ANALISA
Karena fokusnya pada penggunaan energi, maka akan dianalisa:
## Jenis Meteran Yang Digunakan Di Gedung-Gedung

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
meter_types_labels= ['Listrik', 'Air Dingin', 'Uap', 'Air Panas']
sns.countplot(x = 'meter', data = train).set_title("Frek Pengukuran Yang Dilakukan Tiap-Tiap Meteran", fontsize=16)
ax.set_xticklabels(meter_types_labels)
plt.xlabel('Meter Type', fontsize=12)
plt.ylabel('Readings Count', fontsize=12)
plt.show()

**observasi**: Meteran listrik adalah jenis meteran yang paling banyak di gedung-gedung

## Rata-rata energi yang dikonsumsi dari masing-masing listrik, air hangat, uap, dan air panas pada seluruh gedung

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
meter_types_labels= ['Listrik', 'Air Dingin', 'Uap', 'Air Panas']
meter_types = train.groupby('meter').meter_reading.mean().reset_index()
sns.barplot(x ='meter', y ='meter_reading', data = meter_types).set_title("Rata-Rata Penggunaan Energi Yang Dibaca Tiap-Tiap Meter Type", fontsize=16)
ax.set_xticklabels(meter_types_labels)
plt.xlabel('Meter Type', fontsize=12)
plt.ylabel('Rata-Rata', fontsize=12)
plt.show()

**Observasi**: Uap menggunakan energi paling banyak

## Rata-rata penggunaan energi tiap gedung

In [ ]:
building_groups = train.groupby(['building_id']).meter_reading.mean().reset_index()

plt.figure(figsize=(8,8))
sns.lineplot(x = building_groups['building_id'], y = building_groups.meter_reading).set_title("Rata-Rata Pengukuran Energi Yang Dilakukan Tiap Gedung", fontsize=16)
plt.xlabel('Building Id', fontsize=12)
plt.ylabel('Average Log Meter Reading', fontsize=12)
plt.xticks(rotation=45)
plt.show()

## Rata-Rata Penggunaan Energi Perjam dan Perbulan

In [ ]:
hour_groups = train.groupby(['hour']).meter_reading.mean().reset_index()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,8))
fig.suptitle('Rata-Rata Penggunaan Energi')
sns.lineplot(x = hour_groups['hour'], y = hour_groups['meter_reading'], color='green', ax = ax1)
ax1.set_title('Rata-Rata Penggunaan Energi Per Jam')
month_groups = train.groupby(['month']).meter_reading.mean().reset_index()
sns.lineplot(x = month_groups['month'], y = month_groups['meter_reading'], color='purple', ax = ax2)
ax2.set_title('Rata-Rata Penggunaan Energi Per Bulan')

plt.show()

**observasi**: Energi paling banyak digunakan di pukul 10-15 yang mana waktu tersebut merupakan waktu produktif (waktu kerja)
## Rata-Rata Penggunaan Energi Berdasarkan Primary Use

In [ ]:
prim_use = train.groupby(['primary_use'])
prim_use = prim_use['meter_reading'].mean().reset_index()

prim_use

**observasi**: Gedung Serbaguna menggunakan energi paling tinggi

# HASIL PREPROCESSING

In [ ]:
test

In [ ]:
train

In [ ]:
test.to_csv('EDA_KELOMPOKE.csv', index=False)

In [ ]:
train.to_csv('EDA_KELOMPOKE_TRAIN.csv', index=False)